<a href="https://colab.research.google.com/github/fallensoapbubble/NLPGuide/blob/main/MCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For projects using pip for dependencies:[py sdk](https://github.com/modelcontextprotocol/python-sdk)


| Capability | Controlled By | Direction              | Side Effects        | Approval Needed | Typical Use Cases                     |
|------------|---------------|------------------------|---------------------|-----------------|---------------------------------------|
| Tools      | Model (LLM)   | Client → Server        | Yes (potentially)   | Yes             | Actions, API calls, data manipulation |
| Resources  | Application   | Client → Server        | No (read-only)      | Typically no    | Data retrieval, context gathering     |
| Prompts    | User          | Server → Client        | No                  | No              | Guided workflows, specialized templates |
| Sampling   | Server        | Server → Client → Server | Indirectly        | Yes             | Multi-step tasks, agentic behaviors   |




This guide uses **`FastMCP`**, but I will explain exactly how it relies on **FastAPI** and **JSON-RPC** under the hood. This satisfies your goal of learning the "Abstract" (Concepts) and "Below the Wrapper" (Internal mechanics).

### The Abstract: What is MCP?


Think of MCP (Model Context Protocol) as a **universal menu**.

* **The Server (Your Code):** holds the menu (list of tools) and the kitchen (the functions).
* **The Client (AI/Claude):** reads the menu and orders items.
* **The Protocol:** The specific language they use to talk (JSON-RPC).

---

### Step 1: Install Dependencies

`FastMCP` is a wrapper around `FastAPI` (for the web server) and `Pydantic` (for data validation).

**Run this cell:**



In [13]:
!pip install fastmcp uvicorn requests


---

### Step 2: The Setup (Wrapper vs. Reality)

Here we create the server.

**The Wrapper View:** You just type `FastMCP("Name")`.
**Below the Wrapper (What happens inside):**

1. `FastMCP` initializes a **FastAPI** application internally.
2. It sets up specific routes (URLs) that the MCP standard requires, specifically an **SSE endpoint** (Server-Sent Events) at `/sse` and a message handler at `/messages`.
3. It prepares to listen for **JSON-RPC** requests (a standard way to send commands like `{"method": "call_tool", ...}`).


**Run this cell:**


In [15]:
import threading
from fastmcp import FastMCP

# This creates a FastAPI app internally!
# It automatically sets up routes to handle AI connections.
mcp = FastMCP("My Demo Server")


---

### Step 3: Defining a Tool (The Translation Layer)

You write simple Python. `FastMCP` converts this into a schema the AI can understand.

**The Wrapper View:** You use the decorator `@mcp.tool()`.
**Below the Wrapper (What happens inside):**

1. **Introspection:** `FastMCP` looks at your function signature (`a: int, b: int`).
2. **Schema Generation:** It uses **Pydantic** to turn that signature into a JSON Schema definition.
3. **Registration:** It saves this schema in a registry. When an AI connects later, it sends this JSON so the AI knows "add" takes two integers.

**Run this cell:**


In [16]:
@mcp.tool()
def add(a: int, b: int) -> int:
    """Adds two numbers together."""
    return a + b

print("✅ Tool registered. Internal FastAPI routes updated.")

✅ Tool registered. Internal FastAPI routes updated.



---

### Step 4: Running the Server (The Threading Fix)

This is where we solve your previous error.

**Why the Error Happened:** `FastMCP` (via Uvicorn) wants to control the "Main Loop" of the program. Google Colab *also* wants to control the Main Loop. They fight, and you get `RuntimeError`.
**The Fix:** We push the server into a background thread.

**Run this cell:**


In [17]:
import uvicorn

# We define a function to start the server
def start_server():
    # 'sse' is the transport layer.
    # It tells FastAPI to open a persistent connection for the AI.
    mcp.run(transport="sse", port=8000)

# We start that function in a SEPARATE thread
# This creates a parallel lane so it doesn't crash Colab's main lane.
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

print("✅ Server running on http://localhost:8000")


✅ Server running on http://localhost:8000




╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │
│                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │
│                                                                              │
│                                FastMCP 2.14.1                                │
│                                                                              │
│                                                                              │
│                  🖥  Server name: My Demo Server                              │
│                                                                              │
│                  📦 Transport:   SSE                                         │
│                  🔗 Server URL:  http://127.0.0.1:8000/sse                   │
│                           